In [1]:
import os
import cv2
import numpy as np
import tempfile
import shutil


In [2]:
import tensorflow as tf

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [4]:
# Function to load images from a directory
def load_images_from_directory(directory):
    images = []
    for filename in os.listdir(directory):
        image_path = os.path.join(directory, filename)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (100, 100))  # Resize images to a common size (adjust as needed)
        images.append(image)
    return np.array(images)

In [5]:
# Load images from directories and create labels
with_mask_images = load_images_from_directory("/kaggle/input/face-mask-dataset/data/with_mask")
without_mask_images = load_images_from_directory("/kaggle/input/face-mask-dataset/data/without_mask")


In [6]:
print("Shape of all_images:", with_mask_images.shape)
print("Shape of all_images_without:", without_mask_images.shape)


Shape of all_images: (3725, 100, 100, 3)
Shape of all_images_without: (3828, 100, 100, 3)


In [7]:

with_mask_labels = np.ones(with_mask_images.shape[0])
without_mask_labels = np.zeros(without_mask_images.shape[0])



In [8]:
# Combine the images and create labels
all_images = np.concatenate((with_mask_images, without_mask_images), axis=0)
all_labels = np.concatenate((with_mask_labels, without_mask_labels), axis=0)

In [9]:
# Normalize image data to range [0, 1]
all_images = all_images.astype('float32') / 255.0


In [10]:
# One-hot encode the labels
all_labels = to_categorical(all_labels)

In [11]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.2, random_state=42)


In [12]:
# Build the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))  # 2 classes: With Mask and Without Mask







In [13]:
# Model Compilation
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])




In [14]:
# Model Training
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
189/189 [==============================] - 67s 347ms/step - loss: 0.3977 - accuracy: 0.8203 - val_loss: 0.2681 - val_accuracy: 0.8981
Epoch 2/10
189/189 [==============================] - 65s 344ms/step - loss: 0.2611 - accuracy: 0.8954 - val_loss: 0.2116 - val_accuracy: 0.9093
Epoch 3/10
189/189 [==============================] - 65s 343ms/step - loss: 0.2112 - accuracy: 0.9194 - val_loss: 0.1596 - val_accuracy: 0.9385
Epoch 4/10
189/189 [==============================] - 65s 342ms/step - loss: 0.1795 - accuracy: 0.9315 - val_loss: 0.1841 - val_accuracy: 0.9391
Epoch 5/10
189/189 [==============================] - 65s 342ms/step - loss: 0.1529 - accuracy: 0.9454 - val_loss: 0.1390 - val_accuracy: 0.9497
Epoch 6/10
189/189 [==============================] - 65s 343ms/step - loss: 0.1280 - accuracy: 0.9535 - val_loss: 0.1340 - val_accuracy: 0.9550
Epoch 7/10
189/189 [==============================] - 65s 344ms/step - loss: 0.1216 - accuracy: 0.9595 - val_loss: 0.1358 - val_ac

In [15]:
model_path = 'C:\\Users\\FAITHY\\Documents\\newmodel_model.h5'


In [16]:
model.save(model_path)

In [17]:
from tensorflow.keras.models import load_model


In [18]:
loaded_model = load_model(model_path)